In [8]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
import datasets
import joblib
import os
from sklearn.base import clone
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
df = pd.read_csv('/content/drive/MyDrive/Grammar_autocorrection_df1_new.csv')

In [11]:
df.head()

,Error Type,Ungrammatical Statement,Standard English,Ungrammatical Length,Standard Length,Tokenized Ungrammatical,Tokenized Standard,POS_Ungrammatical,POS_Standard,POS_W2V_Ungram,...,W2V_Ungram,W2V_Standard,Bigrams_Ungrammatical,Bigrams_Standard,Trigrams_Ungrammatical,Trigrams_Standard,W2V_Bigrams_Ungrammatical,W2V_Bigrams_Standard,W2V_Trigrams_Ungrammatical,W2V_Trigrams_Standard
0,verb tense errors,i goes to the store everyday,i go to the store everyday,29,27,"['i', 'goes', 'to', 'the', 'store', 'everyday']","['i', 'go', 'to', 'the', 'store', 'everyday']","[('i', 'NN'), ('goes', 'VBZ'), ('to', 'TO'), (...","[('i', 'NN'), ('go', 'VBP'), ('to', 'TO'), ('t...",[ 0.06899698 -0.11690328 0.17168738 0.160985...,...,[-0.177742 0.5408214 0.27710912 0.102996...,[-0.18476208 0.565066 0.28604895 0.110692...,"['i goes', 'goes to', 'to the', 'the store', '...","['i go', 'go to', 'to the', 'the store', 'stor...","['i goes to', 'goes to the', 'to the store', '...","['i go to', 'go to the', 'to the store', 'the ...",[ 5.3249515e-04 1.2377683e-02 2.6096594e-02 ...,[ 0.00782952 0.01257214 0.03556162 0.002165...,[ 0.01734993 -0.00260671 -0.00072663 -0.003998...,[ 0.01177527 -0.00479391 -0.00123065 0.005437...
1,verb tense errors,they was playing soccer last night,they were playing soccer last night,35,36,"['they', 'was', 'playing', 'soccer', 'last', '...","['they', 'were', 'playing', 'soccer', 'last', ...","[('they', 'PRP'), ('was', 'VBD'), ('playing', ...","[('they', 'PRP'), ('were', 'VBD'), ('playing',...",[ 0.08898706 -0.16277023 0.00065081 0.091216...,...,[-0.22198214 0.6069789 0.3287705 0.110492...,[-0.20759511 0.6033656 0.326151 0.119404...,"['they was', 'was playing', 'playing soccer', ...","['they were', 'were playing', 'playing soccer'...","['they was playing', 'was playing soccer', 'pl...","['they were playing', 'were playing soccer', '...",[-0.001242 0.00464096 0.01187895 0.001658...,[ 0.00568041 0.00796309 0.00476048 0.003099...,[-2.4676307e-03 -8.5125500e-03 7.1918243e-04 ...,[ 2.0901330e-03 -5.0771208e-03 -3.5517749e-03 ...
2,verb tense errors,she have completed her homework,she has completed her homework,32,31,"['she', 'have', 'completed', 'her', 'homework']","['she', 'has', 'completed', 'her', 'homework']","[('she', 'PRP'), ('have', 'VBP'), ('completed'...","[('she', 'PRP'), ('has', 'VBZ'), ('completed',...",[-0.05448146 -0.16768712 -0.05421133 0.071171...,...,[-0.18468447 0.556527 0.2670376 0.099153...,[-0.19615619 0.5517267 0.27528045 0.090484...,"['she have', 'have completed', 'completed her'...","['she has', 'has completed', 'completed her', ...","['she have completed', 'have completed her', '...","['she has completed', 'has completed her', 'co...",[ 5.72577212e-03 1.65572483e-03 8.72172974e-...,[ 1.1315383e-03 -5.4916982e-03 1.3190885e-02 ...,[ 0.00492086 -0.00563675 -0.00285741 0.004414...,[-0.01038283 -0.00692296 -0.00054124 0.011133...
3,verb tense errors,he do not know the answer,he does not know the answer,25,27,"['he', 'do', 'not', 'know', 'the', 'answer']","['he', 'does', 'not', 'know', 'the', 'answer']","[('he', 'PRP'), ('do', 'VBZ'), ('not', 'RB'), ...","[('he', 'PRP'), ('does', 'VBZ'), ('not', 'RB')...",[-5.4591924e-02 -1.1197346e-01 -5.1746000e-02 ...,...,[-0.22064425 0.65218335 0.31709093 0.115940...,[-0.20784527 0.6110135 0.2986785 0.110379...,"['he do', 'do not', 'not know', 'know the', 't...","['he does', 'does not', 'not know', 'know the'...","['he do not', 'do not know', 'not know the', '...","['he does not', 'does not know', 'not know the...",[-0.00033541 0.00086477 0.01327611 -0.009891...,[-0.00465144 -0.00187735 0.00977874 -0.007753...,[ 1.1856072e-03 -2.4600534e-03 -2.7238706e-04 ...,[ 4.04477399e-03 -9.81178135e-03 5.62989060e-...
4,verb tense errors,the sun rise in the east,the sun rises in the east,25,26,"['the', 'sun', 'rise', 'in', 'the', 'east']","['the', 'sun', 'rises', 'in', 'the', 'east']","[('the', 'DT'), ('sun', 'NN'), ('rise', 'NN'),...","[('the', 'DT'), ('sun', 'NN'), ('rises', 'VBZ'...",[ 0

In [12]:
# printing dataset columns
print(df.columns)

Index(['Error Type', 'Ungrammatical Statement', 'Standard English',
       'Ungrammatical Length', 'Standard Length', 'Tokenized Ungrammatical',
       'Tokenized Standard', 'POS_Ungrammatical', 'POS_Standard',
       'POS_W2V_Ungram', 'POS_W2V_Standard', 'POS_seq_U', 'POS_seq_S',
       'W2V_Ungram', 'W2V_Standard', 'Bigrams_Ungrammatical',
       'Bigrams_Standard', 'Trigrams_Ungrammatical', 'Trigrams_Standard',
       'W2V_Bigrams_Ungrammatical', 'W2V_Bigrams_Standard',
       'W2V_Trigrams_Ungrammatical', 'W2V_Trigrams_Standard'],
      dtype='object')


# Naive Bayes

In [13]:
embedding_experiments = {
    'POS':     ('POS_W2V_Ungram',             'POS_W2V_Standard'),
    'Unigram': ('W2V_Ungram',                 'W2V_Standard'),
    'Bigram':  ('W2V_Bigrams_Ungrammatical',  'W2V_Bigrams_Standard'),
    'Trigram': ('W2V_Trigrams_Ungrammatical', 'W2V_Trigrams_Standard')
}

In [14]:
# ── 0) Define & apply your parser once ──────────────────────────────────────
def parse_embedding(s):
    if isinstance(s, str):
        return np.array([float(x) for x in s.strip('[]').split()], dtype=float)
    return np.zeros(0, dtype=float)

for ug_col, st_col in embedding_experiments.values():
    df[ug_col] = df[ug_col].apply(parse_embedding)
    df[st_col] = df[st_col].apply(parse_embedding)

# ── 1) Pipeline builder ─────────────────────────────────────────────────────
def make_embedding_pipeline(ug_col, st_col):
    n = len(df)
    # Stack the two columns into one long list of arrays
    feats = list(df[ug_col]) + list(df[st_col])
    y     = np.array([0]*n + [1]*n)               # 0=ungrammatical, 1=correct
    X_df  = pd.DataFrame({'feat': feats})

    # transformer: np.vstack on that 'feat' column → (2n × dim) matrix
    stacker = FunctionTransformer(lambda X: np.vstack(X['feat']), validate=False)

    pipe = Pipeline([
        ('stack', stacker),
        ('clf',   GaussianNB())
    ])
    return pipe, X_df, y

# ── 2) Train/Test split evaluation (90/10) ─────────────────────────────────────────
print("=== Train/Test split 90/10 ===")
for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline(ug, st)

    X_tr, X_te, y_tr, y_te = train_test_split(
        X_df, y,
        test_size=0.1,
        stratify=y,
        random_state=42
    )
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)

    print(f"\n{name} accuracy: {accuracy_score(y_te, preds):.4f}")
    print(classification_report(y_te, preds, digits=4))

# ── 3) 10-fold CV ────────────────────────────────────────────────────────────
print("\n=== 10-Fold CV ===")
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Loop over each embedding experiment
for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline(ug, st)
    accs, precs, recs, f1s = [], [], [], []

    # Perform CV
    for train_idx, test_idx in cv.split(X_df, y):
        p = clone(pipe)
        X_tr, X_te = X_df.iloc[train_idx], X_df.iloc[test_idx]
        y_tr, y_te = np.array(y)[train_idx], np.array(y)[test_idx]

        p.fit(X_tr, y_tr)
        preds = p.predict(X_te)

        accs.append(accuracy_score(y_te, preds))
        precs.append(precision_score(y_te, preds))
        recs.append(recall_score(y_te, preds))
        f1s.append(f1_score(y_te, preds))

    # Print summary
    print(f"{name} - CV Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f} | "
          f"Precision: {np.mean(precs):.4f} ± {np.std(precs):.4f} | "
          f"Recall: {np.mean(recs):.4f} ± {np.std(recs):.4f} | "
          f"F1: {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")

    # Retrain on full data and save the pipeline
    pipe.fit(X_df, y)


=== Train/Test split 90/10 ===

POS accuracy: 0.5149
              precision    recall  f1-score   support

           0     0.5197    0.3911    0.4463       202
           1     0.5119    0.6386    0.5683       202

    accuracy                         0.5149       404
   macro avg     0.5158    0.5149    0.5073       404
weighted avg     0.5158    0.5149    0.5073       404


Unigram accuracy: 0.5272
              precision    recall  f1-score   support

           0     0.5226    0.6287    0.5708       202
           1     0.5342    0.4257    0.4738       202

    accuracy                         0.5272       404
   macro avg     0.5284    0.5272    0.5223       404
weighted avg     0.5284    0.5272    0.5223       404


Bigram accuracy: 0.4975
              precision    recall  f1-score   support

           0     0.4982    0.6881    0.5780       202
           1     0.4960    0.3069    0.3792       202

    accuracy                         0.4975       404
   macro avg     0.4971 

#SVM

In [15]:
from sklearn.svm import SVC

# Make directory to save models
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

def stack_feats(X):
    return np.vstack(X['feat'])

def make_embedding_pipeline_svm(ug_col, st_col):
    n = len(df)
    feats = list(df[ug_col]) + list(df[st_col])
    y     = np.array([0]*n + [1]*n)  # 0=ungrammatical, 1=correct
    X_df  = pd.DataFrame({'feat': feats})
    stacker = FunctionTransformer(stack_feats, validate=False)

    pipe = Pipeline([
        ('stack', stacker),
        ('scaler', StandardScaler()),
        ('clf',   SVC(kernel='linear',class_weight='balanced',probability=True))
    ])
    return pipe, X_df, y

trained_pipelines = {}
print("=== Train/Test split 90/10 with SVM ===")
for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline_svm(ug, st)

    X_tr, X_te, y_tr, y_te = train_test_split(
        X_df, y,
        test_size=0.1,
        stratify=y,
        random_state=42
    )
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)
    trained_pipelines[name] = pipe

    print(f"\n{name} accuracy: {accuracy_score(y_te, preds):.4f}")
    print(classification_report(y_te, preds, digits=4))

    # Save model
    model_path = os.path.join(save_dir, f"{name}_svm_pipeline.pkl")
    joblib.dump(pipe, model_path)
    print(f"Saved model to {model_path}")

=== Train/Test split 90/10 with SVM ===

POS accuracy: 0.5322
              precision    recall  f1-score   support

           0     0.5376    0.4604    0.4960       202
           1     0.5281    0.6040    0.5635       202

    accuracy                         0.5322       404
   macro avg     0.5329    0.5322    0.5298       404
weighted avg     0.5329    0.5322    0.5298       404

Saved model to saved_models/POS_svm_pipeline.pkl

Unigram accuracy: 0.5297
              precision    recall  f1-score   support

           0     0.5312    0.5050    0.5178       202
           1     0.5283    0.5545    0.5411       202

    accuracy                         0.5297       404
   macro avg     0.5298    0.5297    0.5294       404
weighted avg     0.5298    0.5297    0.5294       404

Saved model to saved_models/Unigram_svm_pipeline.pkl

Bigram accuracy: 0.5124
              precision    recall  f1-score   support

           0     0.5130    0.4901    0.5013       202
           1     0.511

In [16]:
print("\n=== 10-Fold Cross Validation with SVM  ===")
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cv_results = {}

for name, (ug, st) in embedding_experiments.items():
    # Prepare input features and labels
    pipe, X_df, y = make_embedding_pipeline_svm(ug, st)

    fold_precisions = []
    fold_recalls = []
    fold_f1s = []
    all_preds = []
    all_true = []
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X_df, y)):
        X_train, X_val = X_df.iloc[train_idx], X_df.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        fold_model = Pipeline([
            ('stack', FunctionTransformer(stack_feats, validate=False)),
            ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))
        ])
        fold_model.fit(X_train, y_train)
        preds = fold_model.predict(X_val)

        all_preds.extend(preds)
        all_true.extend(y_val)

        # Metrics per fold
        acc = accuracy_score(y_val, preds)
        prec = precision_score(y_val, preds, average='binary', zero_division=0)
        rec = recall_score(y_val, preds, average='binary', zero_division=0)
        f1 = f1_score(y_val, preds, average='binary', zero_division=0)

        fold_accuracies.append(acc)
        fold_precisions.append(prec)
        fold_recalls.append(rec)
        fold_f1s.append(f1)

    # Mean and Std for each metric
    mean_acc, std_acc = np.mean(fold_accuracies), np.std(fold_accuracies)
    mean_prec, std_prec = np.mean(fold_precisions), np.std(fold_precisions)
    mean_rec, std_rec = np.mean(fold_recalls), np.std(fold_recalls)
    mean_f1, std_f1 = np.mean(fold_f1s), np.std(fold_f1s)

    # Print summary in desired format
    print(f"{name} - CV Accuracy: {mean_acc:.4f} ± {std_acc:.4f} | "
          f"Precision: {mean_prec:.4f} ± {std_prec:.4f} | "
          f"Recall: {mean_rec:.4f} ± {std_rec:.4f} | "
          f"F1: {mean_f1:.4f} ± {std_f1:.4f}")

    # Save the final model trained on the entire dataset
    pipe.fit(X_df, y)
    model_path = os.path.join(save_dir, f"{name}_CV_svm_pipeline.pkl")
    joblib.dump(pipe, model_path)
    print(f"Saved final CV model to {model_path}")


=== 10-Fold Cross Validation with SVM  ===
POS - CV Accuracy: 0.5203 ± 0.0258 | Precision: 0.5178 ± 0.0223 | Recall: 0.5981 ± 0.0279 | F1: 0.5549 ± 0.0232
Saved final CV model to saved_models/POS_CV_svm_pipeline.pkl
Unigram - CV Accuracy: 0.5191 ± 0.0258 | Precision: 0.5182 ± 0.0249 | Recall: 0.5565 ± 0.0441 | F1: 0.5359 ± 0.0280
Saved final CV model to saved_models/Unigram_CV_svm_pipeline.pkl
Bigram - CV Accuracy: 0.4985 ± 0.0171 | Precision: 0.3866 ± 0.2069 | Recall: 0.4619 ± 0.4395 | F1: 0.3453 ± 0.2997
Saved final CV model to saved_models/Bigram_CV_svm_pipeline.pkl
Trigram - CV Accuracy: 0.5025 ± 0.0138 | Precision: 0.4024 ± 0.2014 | Recall: 0.6371 ± 0.3517 | F1: 0.4869 ± 0.2486
Saved final CV model to saved_models/Trigram_CV_svm_pipeline.pkl


In [17]:
# Reload all models into a new dictionary
loaded_pipelines = {}
for name in embedding_experiments.keys():
    model_variants = {}
    # Load train/test split model
    path_train = os.path.join(save_dir, f"{name}_svm_pipeline.pkl")
    if os.path.exists(path_train):
        model_variants['train'] = joblib.load(path_train)
        print(f"Loaded trained model: {name}")

    # Load final CV model
    path_cv = os.path.join(save_dir, f"{name}_CV_svm_pipeline.pkl")
    if os.path.exists(path_cv):
        model_variants['cv'] = joblib.load(path_cv)
        print(f"Loaded CV model: {name}")

    # Optional: use loaded_pipelines as default in your prediction functions
    loaded_pipelines[name] = model_variants


Loaded trained model: POS
Loaded CV model: POS
Loaded trained model: Unigram
Loaded CV model: Unigram
Loaded trained model: Bigram
Loaded CV model: Bigram
Loaded trained model: Trigram
Loaded CV model: Trigram


In [18]:
def get_pos_features(sentence):
    return np.random.rand(50)

def get_unigram_features(sentence):
    return np.random.rand(100)

def get_bigram_features(sentence):
    return np.random.rand(50)

def get_trigram_features(sentence):
    return np.random.rand(50)

def extract_features_for_models(sentence):
    features = {}

    features['POS'] = get_pos_features(sentence)
    features['Unigram'] = get_unigram_features(sentence)
    features['Bigram'] = get_bigram_features(sentence)
    features['Trigram'] = get_trigram_features(sentence)

    return features

def predict_sentence(sentence, pipelines):
    feats_dict = extract_features_for_models(sentence)
    print(f"Input sentence: {sentence}\n")

    for model_name, model_variants in pipelines.items():
        X_test_df = pd.DataFrame({'feat': [feats_dict[model_name]]})

        # Predict using train/test model
        if 'train' in model_variants:
            pred = model_variants['train'].predict(X_test_df)[0]
            prob = model_variants['train'].predict_proba(X_test_df)[0][1]
            print(f"{model_name} (train): {'Correct' if pred == 1 else 'Incorrect'} (confidence: {prob:.2f})")

        # Predict using CV model
        if 'cv' in model_variants:
            pred = model_variants['cv'].predict(X_test_df)[0]
            prob = model_variants['cv'].predict_proba(X_test_df)[0][1]
            print(f"{model_name} (CV): {'Correct' if pred == 1 else 'Incorrect'} (confidence: {prob:.2f})")


# Example usage:
sentence = input("Enter a sentence to check: ")
predict_sentence(sentence, loaded_pipelines)


Enter a sentence to check: he are good
Input sentence: he are good

POS (train): Correct (confidence: 1.00)
POS (CV): Correct (confidence: 1.00)
Unigram (train): Incorrect (confidence: 0.00)
Unigram (CV): Incorrect (confidence: 0.00)
Bigram (train): Correct (confidence: 0.37)
Bigram (CV): Incorrect (confidence: 1.00)
Trigram (train): Incorrect (confidence: 0.87)
Trigram (CV): Incorrect (confidence: 0.37)
